---
title: 使用 Langfuse 监控 OpenAI 结构化输出
description: 学习如何使用 Langfuse 来监控和追踪 OpenAI 的结构化输出功能
category: 集成教程
---

# 教程：使用 Langfuse 追踪 OpenAI 结构化输出

在本教程中，您将学习如何使用 Langfuse 来监控和追踪 OpenAI 的结构化输出功能。

## 什么是结构化输出？
从非结构化输入生成结构化数据是当今人工智能的核心应用场景。结构化输出使得链式大语言模型调用、UI组件生成和基于模型的评估更加可靠。[结构化输出](https://openai.com/index/introducing-structured-outputs-in-the-api/) 是 OpenAI API 的一项新功能，它基于 JSON 模式和函数调用构建，能够在模型输出中强制执行严格的数据结构模式。

**核心优势：**
- 🎯 **可靠性**：确保模型输出严格遵循预定义的数据结构
- 🔗 **链式调用**：使多个模型调用之间的数据传递更加稳定
- 🎨 **UI生成**：为前端组件提供标准化的数据格式
- 📊 **模型评估**：便于对模型输出进行自动化评估和分析

## 如何在 Langfuse 中追踪结构化输出？
如果您使用 OpenAI Python SDK，可以使用 [Langfuse 的直接替换方案](https://langfuse.com/integrations/model-providers/openai-py) 来获得完整的日志记录，只需更改导入语句即可。通过这种方式，您可以在 Langfuse 中监控 OpenAI 生成的结构化输出。

**导入方式对比：**
```diff
- import openai                    # 原始导入方式
+ from langfuse.openai import openai  # Langfuse 集成导入

其他可选导入方式：
+ from langfuse.openai import OpenAI, AsyncOpenAI, AzureOpenAI, AsyncAzureOpenAI
```

**重要说明：** 仅需修改导入语句，其余代码保持不变，即可自动获得完整的调用追踪和监控功能。



## 步骤 1：初始化 Langfuse
使用您从 Langfuse UI 项目设置中获取的 [API 密钥](https://langfuse.com/faq/all/where-are-langfuse-api-keys) 来初始化 Langfuse 客户端，并将它们添加到您的环境变量中。

**配置说明：**
- 🔑 **API 密钥**：从 Langfuse 控制台获取公钥和私钥
- 🌍 **服务器地址**：选择合适的区域服务器（欧盟或美国）
- 🤖 **OpenAI 密钥**：用于调用 OpenAI 的 API 服务

In [ ]:
# 安装必要的依赖包
# langfuse: 用于模型调用追踪和监控的工具
# openai: OpenAI 官方 Python SDK
# --upgrade: 确保安装最新版本
%pip install langfuse openai --upgrade

In [4]:
import os

# 配置环境变量 - 这些密钥用于身份验证和服务访问
# 从项目设置页面获取您的项目密钥：https://cloud.langfuse.com

# Langfuse 公钥 - 用于标识您的项目（可以公开）
os.environ["LANGFUSE_PUBLIC_KEY"] = ""

# Langfuse 私钥 - 用于身份验证（需要保密）
os.environ["LANGFUSE_SECRET_KEY"] = ""

# Langfuse 服务器地址 - 选择离您最近的区域以获得更好的性能
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 欧盟区域
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 美国区域

# OpenAI API 密钥 - 用于调用 OpenAI 的模型服务
# 从 https://platform.openai.com/api-keys 获取
os.environ["OPENAI_API_KEY"] = ""

## 步骤 2：数学辅导示例

在这个示例中，我们将构建一个数学辅导工具，它将解决数学问题的步骤输出为结构化对象数组。

**应用场景：**
- 📚 **教育应用**：为学生提供逐步解题指导
- 🎯 **个性化学习**：用户可以按自己的节奏逐步学习解题过程
- 🔍 **步骤追踪**：每个解题步骤都可以单独显示和分析
- 💡 **交互式学习**：支持用户在任意步骤暂停和思考

这种设置对于需要将每个步骤单独显示的应用程序非常有用，允许用户按照自己的节奏逐步学习解题过程。

（示例改编自 [OpenAI cookbook](https://cookbook.openai.com/examples/structured_outputs_intro)）

**重要提示：** 虽然 OpenAI 也通过其测试版 API (`client.beta.chat.completions.parse`) 提供结构化输出解析功能，但这种方法目前不允许设置 Langfuse 特定的属性，如 `name`、`metadata`、`userId` 等。请使用下面描述的标准 `client.chat.completions.create` 方法配合 `response_format` 参数的方式。

**两种方法对比：**
- ✅ **推荐方式**：`client.chat.completions.create` + `response_format` - 支持完整的 Langfuse 追踪功能
- ⚠️ **限制方式**：`client.beta.chat.completions.parse` - 功能受限，无法设置追踪属性

In [5]:
# 使用 Langfuse 的直接替换方案，仅通过更改导入即可获得完整的日志记录功能
# 这样，您就可以在 Langfuse 中监控 OpenAI 生成的结构化输出
from langfuse.openai import OpenAI
import json  # 用于处理 JSON 数据的标准库

# 指定要使用的 OpenAI 模型
# gpt-4o-2024-08-06 是支持结构化输出的最新模型版本
openai_model = "gpt-4o-2024-08-06"

# 创建 OpenAI 客户端实例
# 这个客户端会自动集成 Langfuse 的追踪功能
client = OpenAI()

在 `response_format` 参数中，您现在可以通过 `json_schema` 提供 JSON Schema。当使用 `response_format` 并设置 `strict: true` 时，模型的输出将严格遵循提供的模式。

**关键概念解释：**
- 📋 **JSON Schema**：定义数据结构的标准格式，类似于数据的"蓝图"
- 🔒 **strict: true**：启用严格模式，确保输出100%符合定义的结构
- ✅ **结构保证**：模型输出将始终包含所需的字段和数据类型

函数调用保持类似的方式，但通过新的参数 `strict: true`，您现在可以确保为函数提供的模式得到严格遵循。

In [6]:
# 定义数学辅导的系统提示词
# 这个提示词告诉模型如何扮演数学老师的角色
math_tutor_prompt = '''
    你是一个有用的数学辅导老师。你将收到一个数学问题，
    你的目标是输出逐步解决方案以及最终答案。
    对于每个步骤，只需提供输出作为方程式，使用解释字段详细说明推理过程。
'''

def get_math_solution(question):
    """
    获取数学问题的结构化解决方案
    
    参数:
        question (str): 要解决的数学问题
    
    返回:
        包含逐步解决方案的结构化响应
    """
    # 调用 OpenAI API 创建聊天完成请求
    # 使用结构化输出确保返回格式符合预定义的 JSON Schema
    response = client.chat.completions.create(
    model = openai_model,  # 指定使用的模型版本
    messages=[
        {
            # 系统消息：定义AI助手的角色和行为规范
            "role": "system",
            "content": math_tutor_prompt
        },
        {
            # 用户消息：包含具体的数学问题
            "role": "user",
            "content": question
        }
    ],
    # 定义响应格式 - 这是结构化输出的核心配置
    response_format={
        # 指定使用 JSON Schema 来定义输出结构
        "type": "json_schema",
        "json_schema": {
            # Schema 的名称，用于标识这个特定的数据结构
            "name": "math_reasoning",
            # 定义具体的数据结构模式
            "schema": {
                # 根对象类型
                "type": "object",
                # 定义对象的属性
                "properties": {
                    # steps: 解题步骤数组
                    "steps": {
                        "type": "array",  # 数组类型
                        # 数组中每个元素的结构
                        "items": {
                            "type": "object",  # 每个步骤是一个对象
                            "properties": {
                                # 解释字段：说明这一步的推理过程
                                "explanation": {"type": "string"},
                                # 输出字段：这一步的数学表达式或结果
                                "output": {"type": "string"}
                            },
                            # 必需字段：每个步骤都必须包含解释和输出
                            "required": ["explanation", "output"],
                            # 不允许额外属性，确保结构严格
                            "additionalProperties": False
                        }
                    },
                    # final_answer: 最终答案
                    "final_answer": {"type": "string"}
                },
                # 根对象的必需字段
                "required": ["steps", "final_answer"],
                # 不允许额外属性
                "additionalProperties": False
            },
            # 启用严格模式：确保输出100%符合定义的结构
            "strict": True
        }
    }
    )

    # 返回模型生成的消息内容
    # choices[0] 表示选择第一个（通常也是唯一的）响应
    return response.choices[0].message

In [7]:
# 使用示例问题进行测试
# 这是一个一元一次方程，适合演示逐步解题过程
question = "如何解这个方程：8x + 7 = -23"

# 调用我们定义的函数获取结构化解决方案
result = get_math_solution(question)

# 打印原始的 JSON 格式响应内容
# 这展示了模型返回的结构化数据
print("原始 JSON 响应:")
print(result.content)

{"steps":[{"explanation":"We need to isolate the term with the variable, 8x. So, we start by subtracting 7 from both sides to remove the constant term on the left side.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"The +7 and -7 on the left side cancel each other out, leaving us with 8x. The right side simplifies to -30.","output":"8x = -30"},{"explanation":"To solve for x, divide both sides of the equation by 8, which is the coefficient of x.","output":"x = -30 / 8"},{"explanation":"Simplify the fraction -30/8 by finding the greatest common divisor, which is 2.","output":"x = -15 / 4"}],"final_answer":"x = -15/4"}


In [8]:
# 逐步打印解题结果 - 展示如何处理结构化输出

# 将 JSON 字符串解析为 Python 字典
# 这展示了结构化输出的一个重要优势：数据格式可预测且易于处理
result = json.loads(result.content)

# 提取解题步骤数组和最终答案
steps = result['steps']
final_answer = result['final_answer']

# 遍历每个解题步骤并格式化输出
for i in range(len(steps)):
    # 打印步骤编号和解释
    print(f"步骤 {i+1}: {steps[i]['explanation']}\n")
    # 打印该步骤的数学表达式
    print(steps[i]['output'])
    print("\n")

# 打印最终答案
print("最终答案:\n\n")
print(final_answer)

Step 1: We need to isolate the term with the variable, 8x. So, we start by subtracting 7 from both sides to remove the constant term on the left side.

8x + 7 - 7 = -23 - 7


Step 2: The +7 and -7 on the left side cancel each other out, leaving us with 8x. The right side simplifies to -30.

8x = -30


Step 3: To solve for x, divide both sides of the equation by 8, which is the coefficient of x.

x = -30 / 8


Step 4: Simplify the fraction -30/8 by finding the greatest common divisor, which is 2.

x = -15 / 4


Final answer:


x = -15/4


## 步骤 3：在 Langfuse 中查看您的追踪记录

现在您可以在 Langfuse 中查看追踪记录和 JSON Schema。

**Langfuse 追踪功能的价值：**
- 📊 **完整记录**：记录每次 API 调用的详细信息
- 🔍 **结构分析**：可视化展示 JSON Schema 和实际输出
- ⏱️ **性能监控**：追踪响应时间和 token 使用情况
- 🐛 **调试支持**：帮助识别和解决问题
- 📈 **使用统计**：分析模型使用模式和成本

[Langfuse 中的示例追踪记录](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/3ecc3849-66c9-4eaf-b26b-bde26b7eebed)

![在 Langfuse UI 中查看示例追踪记录](https://langfuse.com/images/cookbook/integration-openai-structured-outputs-tracing.png)

**图片说明：** 上图展示了 Langfuse 界面中的追踪记录，您可以看到：
- 🔄 **调用流程**：完整的 API 调用链路
- 📝 **输入输出**：系统提示、用户问题和模型响应
- 📊 **结构化数据**：JSON Schema 和实际输出的对比
- ⏱️ **性能指标**：响应时间、token 消耗等关键指标

## 替代方案：使用 SDK 的 `parse` 辅助方法

新版本的 SDK 添加了 `parse` 辅助方法，允许您使用自己的 Pydantic 模型而无需定义 JSON Schema。

**Pydantic 方法的优势：**
- 🐍 **Python 原生**：使用 Python 类定义数据结构，更符合 Python 开发习惯
- 🔧 **类型检查**：提供更好的 IDE 支持和类型提示
- 📝 **简洁语法**：相比 JSON Schema，代码更简洁易读
- ✅ **自动验证**：Pydantic 提供强大的数据验证功能

**注意：** 这种方法目前在 Langfuse 追踪中的功能可能有限。

In [9]:
# 导入 Pydantic 用于定义数据模型
# Pydantic 是 Python 中最流行的数据验证库
from pydantic import BaseModel

# 定义数学推理的数据模型
class MathReasoning(BaseModel):
    """数学推理结果的数据模型"""
    
    # 内嵌类定义单个解题步骤的结构
    class Step(BaseModel):
        """单个解题步骤的数据模型"""
        explanation: str  # 解释说明：描述这一步的推理过程
        output: str       # 输出结果：这一步的数学表达式或计算结果

    # 主模型的字段定义
    steps: list[Step]    # 解题步骤列表：包含所有解题步骤
    final_answer: str    # 最终答案：问题的最终解答

def get_math_solution(question: str):
    """
    使用 Pydantic 模型获取数学问题的结构化解决方案
    
    参数:
        question (str): 要解决的数学问题
    
    返回:
        包含解析后的结构化数据的响应
    """
    # 使用 beta API 的 parse 方法
    # 这个方法会自动将 Pydantic 模型转换为 JSON Schema
    response = client.beta.chat.completions.parse(
        model=openai_model,  # 指定模型
        messages=[
            # 系统消息：定义AI的角色
            {"role": "system", "content": math_tutor_prompt},
            # 用户消息：具体的数学问题
            {"role": "user", "content": question},
        ],
        # 直接传入 Pydantic 模型类，SDK 会自动处理转换
        response_format=MathReasoning,
    )

    # 返回响应消息
    # 注意：使用 .parsed 属性可以直接获取解析后的 Pydantic 对象
    return response.choices[0].message

In [10]:
# 调用函数并获取解析后的 Pydantic 对象
# .parsed 属性直接返回类型化的 Python 对象，而不是 JSON 字符串
result = get_math_solution(question).parsed

# 打印解题步骤 - 这里 result.steps 是 Step 对象的列表
print("解题步骤:")
print(result.steps)

# 打印最终答案 - 直接访问对象属性
print("\n最终答案:")
print(result.final_answer)

[Step(explanation='To isolate the term with the variable on one side of the equation, start by subtracting 7 from both sides.', output='8x = -23 - 7'), Step(explanation='Combine like terms on the right side to simplify the equation.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -30 / 8'), Step(explanation='Simplify the fraction by dividing both the numerator and the denominator by their greatest common divisor, which is 2.', output='x = -15 / 4')]
Final answer:
x = -15/4


## 在 Langfuse 中查看您的追踪记录

现在您可以在 Langfuse 中查看追踪记录和您提供的 Pydantic 模型。

**Pydantic 方法的追踪特点：**
- 🏗️ **模型结构**：显示 Pydantic 模型的类定义和字段类型
- 🔄 **自动转换**：展示从 Pydantic 到 JSON Schema 的自动转换过程
- 📊 **类型信息**：保留完整的 Python 类型注解信息

[Langfuse 中的 Pydantic 示例追踪记录](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/59c4376a-c8eb-4ecb-8780-2f028b87e7eb)

![在 Langfuse UI 中查看 Pydantic 方法的追踪记录](https://langfuse.com/images/cookbook/integration_openai_structured_outputs_tracing_parse.png)

**图片说明：** 上图展示了使用 Pydantic 方法时的 Langfuse 追踪界面，您可以看到：
- 🐍 **Pydantic 模型**：完整的类定义和字段类型信息
- 🔄 **Schema 转换**：自动生成的 JSON Schema
- 📝 **类型安全**：强类型的数据结构和验证
- ⚡ **开发效率**：更简洁的代码和更好的 IDE 支持

## 反馈与支持

如果您有任何反馈或请求，请创建 GitHub [Issue](https://langfuse.com/issue) 或在 [Discord](https://langfuse.com/discord) 上与社区分享您的想法。

**学习总结：**
通过本教程，您学会了：
- 🎯 **结构化输出**：如何使用 OpenAI 的结构化输出功能确保数据格式的一致性
- 📊 **监控追踪**：如何使用 Langfuse 监控和分析 AI 模型的调用过程
- 🔧 **两种方法**：JSON Schema 和 Pydantic 两种定义数据结构的方式
- 💡 **实际应用**：数学辅导场景中的逐步解题展示

**下一步建议：**
- 尝试在您自己的项目中集成 Langfuse 追踪功能
- 探索更复杂的数据结构和验证规则
- 使用 Langfuse 的分析功能优化您的 AI 应用性能